# Dependencies and Variables

In [4]:
import sys
import math
import base64
import argparse
from io import BytesIO
from typing import List

import tqdm
import torch
import pandas as pd

import torchvision
import webdataset as wds
from PIL import Image as IMG
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from IPython.display import Image, HTML, Markdown

sys.path.append("./")
sys.path.append("../")

from models.open_CLIP import OpenCLIP
from models.open_CLIP_adapter import OpenCLIPAdapter
from models.open_clip_wrapper import OpenCLIPWrapper

ModuleNotFoundError: No module named 'lightning'

In [ ]:
gpu = 4
batch_size = 100
model_path=""
open_clip = True
translation = 'google'
adapter = None
dataset_path = "/hadatasets/clip_pt/images/imagenet"
num_workers = 12

In [ ]:
# %env PYTHONPATH=${PYTHONPATH}:/work/diego.moreira/CLIP-PtBr/clip_pt/src
%env TRANSFORMERS_CACHE=/work/diego.moreira/hf_dir
%env HF_HOME=/work/diego.moreira/hf_dir

# Auxiliar Methods

In [9]:
def compute_image_embeddings(image_dataset, model):
    dataloader = DataLoader(image_dataset, batch_size=batch_size, num_workers=num_workers)
    all_embeddings = None
    with torch.no_grad(), torch.cuda.amp.autocast():
        for batch in tqdm.tqdm(dataloader, desc="Encoding images"):
            batch = batch[0].to("cuda")
            embeddings = model.encode_visual(batch)
            if all_embeddings is None:
                all_embeddings = embeddings
            else:
                all_embeddings = torch.cat((all_embeddings, embeddings), dim=0)
    return all_embeddings

def compute_text_embeddings(text_dataset, collate, model):
    dataloader = DataLoader(text_dataset, batch_size=batch_size, num_workers=num_workers, collate_fn=collate)
    all_embeddings = None
    with torch.no_grad(), torch.cuda.amp.autocast():
        for batch in dataloader:
            batch = batch.to("cuda")
            embeddings = model.encode_text(batch)
            if all_embeddings is None:
                all_embeddings = embeddings
            else:
                all_embeddings = torch.cat((all_embeddings, embeddings), dim=0)
    return all_embeddings

def run_evaluation(image_embeddings, text_embeddings, num_prompts=1, images_per_class=50, num_classes=1000):
    # Prompt ensembles are averaged for the final prompt embedding
    if num_prompts > 1:
        text_embeddings = text_embeddings.view(len(text_embeddings)//num_prompts, num_prompts, -1)
        text_embeddings = torch.mean(text_embeddings, dim=1)
        text_embeddings = F.normalize(text_embeddings, dim=1)

    image_embeddings = image_embeddings.to(device=text_embeddings.device, dtype=text_embeddings.dtype)

    scores = image_embeddings @ text_embeddings.t()
    target = torch.arange(0, len(text_embeddings)).repeat_interleave(images_per_class, 0).to("cuda")

    acc1, acc5 = compute_accuracy(scores, target, topk=(1, 5))

    top1 = (acc1 / (num_classes*images_per_class))
    top5 = (acc5 / (num_classes*images_per_class))

    return top1, top5

def get_data(transformation, tokenizer, labels, prompts, idxs):
    image_dataset = torchvision.datasets.ImageNet(
        dataset_path,
        split="val",
        loader=IMG.open,
        transform=transformation
    )

    text_collate = Collator(tokenizer)
    text_datasets = []
    
    ds = BabelImageNetTextDataset(labels=labels, prompts=prompts, class_idxs=idxs)
    text_datasets.append(ds)

    return image_dataset, text_datasets, text_collate

class BabelImageNetTextDataset(Dataset):
    def __init__(self, labels, prompts, class_idxs):
        self.class_idxs = class_idxs
        self.num_prompts = len(prompts)
        self.data = [prompt.format(label) for label in labels for prompt in prompts]

    def __getitem__(self, item):
        return self.data[item]

    def __len__(self):
        return len(self.data)

class Collator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, examples):
        text_features = self.tokenizer(examples)
        return text_features

def compute_accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

# Labels and Prompts

In [5]:
imagenet_prompts_pt = [
    "Uma foto ruim de um  {} .",
    "Uma foto de muitos  {} .",
    "Uma escultura de um  {} .",
    "Uma foto do difícil de ver  {} .",
    "Uma foto de baixa resolução do  {} .",
    "Uma representação de  {} .",
    "Graffiti de um  {} .",
    "Uma foto ruim do  {} .",
    "Uma foto cortada do  {} .",
    "Uma tatuagem de um  {} .",
    "O bordado  {} .",
    "Uma foto de um difícil de ver  {} .",
    "Uma foto brilhante de um  {} .",
    "Uma foto de um  {}  limpo.",
    "Uma foto de um  {}  sujo",
    "Uma foto escura do  {} .",
    "Um desenho de um {} .",
    "Uma foto da minha \" {} \".",
    "O plástico  {} .",
    "Uma foto do legal do  {} .",
    "Uma foto de perto de um  {} .",
    "Uma foto em preto e branco do  {} .",
    "Uma pintura do  {} .",
    "Uma pintura de um  {} .",
    "Uma foto pixelada do  {} .",
    "Uma escultura do  {} .",
    "Uma foto brilhante do  {} .",
    "Uma foto cortada de um  {} .",
    "Um plástico  {} .",
    "Uma foto do  {}  sujo.",
    "Uma foto corrompida de um jpeg de um {} .",
    "uma foto borrada do  {} .",
    "uma foto do  {} .",
    "Uma boa foto do  {} .",
    "uma representação do  {} .",
    "Um  {}  em um jogo de videgame.",
    "Uma foto de um  {} .",
    "Um rabisco de um  {} .",
    "Uma foto de perto do  {} .",
    "Uma foto de um  {} .",
    "O origami. {}",
    "O  {}  em um jogo de videogame.",
    "Um esboço de um  {} .",
    "Um rabisco do  {} .",
    "Um origami. {}",
    "Uma foto de baixa resolução de um {} .",
    "O brinquedo {}  .",
    "Uma representação do  {} .",
    "Uma foto do  {}  limpo.",
    "Uma foto de uma grande  {} .",
    "Uma representação de um  {} .",
    "Uma foto de uma linda  {} .",
    "Uma foto de um  {}  estranho.",
    "Uma foto borrada de um  {}  .",
    "Um desenho animado. {}",
    "Uma arte de um  {} .",
    "Um esboço do  {} .",
    "Uma  {}  bordada.",
    "uma foto pixelada de um  {} .",
    "Itap do  {} .",
    "Uma foto jpeg corrompida do  {} .",
    "Uma boa foto de um  {} .",
    "Um pelúcia  {}",
    "Uma foto do bom  {} .",
    "uma foto da pequena  {} .",
    "Uma foto da estranha  {} .",
    "O desenho animado. {}",
    "Arte do {} ",
    "Um desenho do  {} .",
    "Uma foto da grande  {} .",
    "Uma foto em preto e branco de um  {} .",
    "O pelúcia. {}",
    "Uma foto escura de um  {} .",
    "Itap de um  {} .",
    "Graffiti do  {} .",
    "Um brinquedo {} .",
    "Itap de um  {}",
    "Uma foto um  {}  legal.",
    "Uma foto de um pequeno  {} .",
    "Uma tatuagem do  {} ."
  ]

In [6]:
imagenet_labels_pt = [
    "tenca",
    "peixe dourado",
    "tubarão branco",
    "tubarão tigre",
    "tubarão martelo",
    "raia eletrica",
    "raia stingrays",
    "galo",
    "galinha",
    "avestruz",
    "tentilhão-montês",
    "pintassilgo",
    "pintarroxo-caseiro",
    "junco",
    "passerina cyanea",
    "tordo-americano",
    "bulbo",
    "gaio",
    "pega",
    "poecile",
    "dipper americano",
    "milhafre",
    "águia-de-cabeça-branca",
    "abutre",
    "coruja-lapónica",
    "salamandra-de-fogo",
    "salamandra lisa",
    "tritão",
    "ambystoma maculatum",
    "axolote",
    "rã-touro americana",
    "sapo de árvore",
    "ascaphus",
    "tartaruga marinha comum",
    "tartaruga marinha de couro",
    "tartaruga-de-lama",
    "tartaruga terrapina",
    "tartaruga-de-caixa-da-flórida",
    "coleonyx mitratus ",
    "iguana-verde",
    "anolis carolinensis",
    "aspidoscelis uniparens",
    "agama",
    "lagarto-de-gola",
    "elgaria",
    "monstro-de-gila",
    "lagarto verde europeu",
    "camaleão",
    "dragão de komodo",
    "crocodilo do nilo",
    "jacaré americano",
    "triceratops",
    "carphophis",
    "cobra de pescoço anelado",
    "heterodon platyrhinos",
    "cobra verde lisa",
    "cobra-real",
    "cobras-liga",
    "cobra d'água",
    "cobra-cipó",
    "cobra da noite",
    "jibóia",
    "píton africana",
    "cobra indiana",
    "mamba verde",
    "cobra do mar",
    "víbora-de-chifres",
    "cascavel-diamante oriental",
    "cascavel-chifruda",
    "trilobita",
    "opilião",
    "escorpião",
    "aranha de jardim amarela",
    "aranha do celeiro",
    "aranha de jardim europeia",
    "viúva negra",
    "tarântula",
    "aranhas-lobo",
    "carrapato",
    "centopéia",
    "tetraz-lira",
    "lagópode",
    "tetraz-de-colar",
    "tetraz-das-pradarias",
    "pavão",
    "codorna",
    "perdiz",
    "papagaio cinzento africano",
    "arara",
    "cacatua-de-crista-amarela",
    "loris arco-Íris",
    "centropus",
    "abelharuco",
    "bucerotiformes",
    "beija flor",
    "galbulidae",
    "tucano",
    "pato",
    "merganso-de-poupa",
    "ganso",
    "cisne negro",
    "elefante",
    "equidna",
    "ornitorrinco",
    "canguru pequeno",
    "coala",
    "vombate",
    "água-viva",
    "anêmona do mar",
    "coral cerebral",
    "tênia",
    "nematóide",
    "concha",
    "caracol",
    "lesma",
    "lesma marinha",
    "chiton",
    "náutilo de câmara",
    "caranguejo dungeness",
    "caranguejo da pedra",
    "caranguejo violinista",
    "caranguejo real vermelho",
    "lagosta americana",
    "lagosta espinhosa",
    "lagostim",
    "caranguejo eremita",
    "isópode",
    "cegonha branca",
    "cegonha negra",
    "colhereiro",
    "flamingo",
    "pequena garça azul",
    "grande garça",
    "pássaro abetouro",
    "grou",
    "carão",
    "galinha-dágua-americana",
    "galeirão americano",
    "abetarda",
    "rola-do-mara",
    "pilrito-comum",
    "perna vermelha comum",
    "maçarico-de-bico-comprido",
    "ostraceiro",
    "pelicano",
    "pinguim rei",
    "albatroz",
    "baleia cinzenta",
    "orca",
    "dugongo",
    "leão marinho",
    "chihuahua",
    "spaniel japonês",
    "maltês",
    "pequinês",
    "shih tzu",
    "rei carlos spaniel",
    "spaniel anão continental",
    "toy terrier",
    "rhodesian ridgeback",
    "galgo afegão",
    "basset hound",
    "beagle",
    "bloodhound",
    "bluetick coonhound",
    "coonhound preto e castanho",
    "treeing walker coonhound",
    "foxhound inglês",
    "redbone coonhound",
    "borzoi",
    "lébrel irlandês",
    "galgo italiano",
    "galgo",
    "podengo ibicenco",
    "elkound norueguês",
    "otterhound",
    "saluki",
    "deerhound",
    "weimaraner",
    "staffordshire bull terrier",
    "american staffordshire terrier",
    "bedlington terrier",
    "border terrier",
    "kerry blue terrier",
    "terrier irlandês",
    "norfolk terrier",
    "norwich terrier",
    "Yorkshire terrier",
    "fox terrier pelo de arame",
    "lakeland terrier",
    "sealyham terrier",
    "airedale terrier",
    "cairn terrier",
    "terrier australiano",
    "dandie dinmont terrier",
    "boston terrier",
    "schnauzer miniatura",
    "schnauzer gigante",
    "schnauzer",
    "terrier escocês",
    "terrier tibetano",
    "silky terrier australiano",
    "soft-coated wheaten terriers",
    "west highland white terrier",
    "lhasa apso",
    "flat-coated retriever",
    "curly-coated retriever",
    "golden retriever",
    "labrador retriever",
    "chesapeake bay retriever",
    "braco alemão de pelo curto",
    "braco húngaro de pelo curto",
    "setter inglês",
    "setter irlandês",
    "setter gordon",
    "spaniel bretão",
    "clumber spaniel",
    "spring spaniel inglês",
    "springer spaniel de gales",
    "cocker spaniel inglês",
    "sussex spaniel",
    "cão dágua irlandês",
    "kuvasz",
    "schipperke",
    "pastor belga groenendael",
    "pastor belga malinois",
    "pastor de brie",
    "kelpie australiano",
    "komondor",
    "old english sheepdog",
    "pastor de shetland",
    "collie",
    "border collie",
    "boiadeiro da flandres",
    "rotweiller",
    "cão pastor alemão",
    "dobermann",
    "pincher miniatura",
    "grande boiadeiro suíço",
    "boiadeiro de berna",
    "boiadeiro de appenzell",
    "boiadeiro de entlebuch",
    "boxer",
    "bullmastiff",
    "mastim tibetano",
    "bulldog francês",
    "dogue alemão",
    "cão são bernardo",
    "husky",
    "malamute-do-alasca",
    "husky siberiano",
    "dálmata",
    "affenpinscher",
    "basenji",
    "pug",
    "leonberger",
    "cão terra nova",
    "cão de montanha dos pirenéus",
    "samoieda",
    "spitz alemão anão",
    "chow chow",
    "keeshond",
    "griffon de bruxelas",
    "welsh corgi pembroke",
    "welsh corgi cardigan",
    "poodle toy",
    "poodle miniatura",
    "poodle padrão",
    "pelado mexicano Xoloitzcuintle,",
    "lobo cinzento",
    "lobo de tundra do alasca",
    "lobo vermelho ou lobo-guará",
    "coiote",
    "dingo",
    "cão selvagem asiático",
    "cão selvagem africano",
    "hiena",
    "raposa vermelha",
    "raposa anã",
    "raposa do ártico",
    "raposa cinzenta",
    "gato malhado",
    "toyger",
    "gato persa",
    "gato siamês",
    "mau egípcio",
    "puma",
    "lince",
    "leopardo",
    "leopardo das neves",
    "jaguar",
    "leão",
    "tigre",
    "guepardo",
    "urso marrom",
    "urso negro americano",
    "urso polar",
    "urso-beiçudo",
    "herpestidae",
    "suricata",
    "cicindelas",
    "joaninha",
    "carabidae",
    "serra-pau",
    "chrysomelidae",
    "besouro do esterco",
    "gorgulhos",
    "berbigão",
    "mosca",
    "abelha",
    "formiga",
    "gafanhoto",
    "grilo",
    "bicho-pau",
    "barata",
    "louva-deus",
    "cigarra",
    "cigarrinhas",
    "neuroptera",
    "libélula",
    "donzelinhas",
    "vanessa atalanta",
    "aphantopus hyperantus",
    "borboleta monarca",
    "pequena borboleta branca",
    "phoebis sennae",
    "licenídea",
    "estrela do mar",
    "ouriço-do-mar",
    "pepino do mar",
    "coelho de cauda de algodão",
    "lebre",
    "coelho angorá",
    "hamster",
    "porco-espinho",
    "esquilo raposa",
    "marmota",
    "castor",
    "porquinho da índia",
    "cavalo alazão comum",
    "zebra",
    "porco",
    "javali selvagem",
    "javali",
    "hipopótamo",
    "boi",
    "búfalo marinho",
    "búfalo",
    "carneiro (ovelha macho adulto)",
    "carneiro selvagem",
    "íbex alpino",
    "hartebeest",
    "impala (antílope)",
    "gazela",
    "camelo árabe",
    "lhama",
    "doninha",
    "vison",
    "doninha europeia",
    "furão de patas negras",
    "lontra",
    "gambá",
    "texugo",
    "tatu",
    "preguiça de três dedos",
    "orangotango",
    "gorila",
    "chimpanzé",
    "gibão",
    "siamang",
    "cercopithecus",
    "macaco de patas",
    "babuíno",
    "macaco",
    "semnopithecus",
    "colobus preto e branco",
    "macaco-narigudo",
    "sagui",
    "capuchinho de cabeça branca",
    "bugio",
    "macaco titi",
    "macaco aranha de geoffroy",
    "macaco-esquilo comum",
    "lémure-de-cauda-anelada",
    "indri",
    "elefante asiático",
    "elefante do mato africano",
    "panda vermelho",
    "panda gigante",
    "thyrsites atun",
    "enguia",
    "salmão prateado",
    "peixe-soldado",
    "peixe-palhaço",
    "esturjão",
    "peixe-jacaré",
    "peixe-leão",
    "peixe baiacu",
    "ábaco",
    "abaya",
    "vestido acadêmico",
    "acordeão",
    "violão",
    "porta-aviões",
    "avião",
    "dirigível",
    "altar",
    "ambulância",
    "veículo anfíbio",
    "relógio analógico",
    "apiário",
    "avental",
    "cesto de lixo",
    "espingarda de assalto",
    "mochila",
    "padaria",
    "trave olímpica",
    "balão",
    "caneta esferográfica",
    "curativo",
    "banjo",
    "balaústre",
    "barra",
    "cadeira de barbeiro",
    "barbearia",
    "celeiro",
    "barômetro",
    "barril",
    "carrinho de mão",
    "beisebol",
    "basquetebol",
    "berço",
    "fagote",
    "touca de natação",
    "toalha de banho",
    "banheira",
    "carrinha",
    "farol",
    "taça",
    "chapéu militar pele de urso ou barretina",
    "garrafa de cerveja",
    "copo de cerveja",
    "torre sineira",
    "babador de bebê",
    "bicicleta tandem",
    "bikini",
    "encadernador",
    "binóculos",
    "casa de passarinho",
    "barco",
    "bob trenó",
    "bolo tie",
    "poke bonnet",
    "estante",
    "livraria",
    "tampa de garrafa",
    "arco de caça",
    "gravata-borboleta",
    "placa memorial em latão",
    "sutiã",
    "quebra-mar",
    "peitoral",
    "vassoura",
    "balde",
    "fivela",
    "colete a prova de balas",
    "trem de alta velocidade",
    "açougue",
    "táxi",
    "caldeirão",
    "vela",
    "canhão",
    "canoa",
    "abridor de lata",
    "casaco",
    "espelho do carro",
    "carrossel",
    "ferramentas",
    "caixa de papelão",
    "roda de carro",
    "caixa eletrônico",
    "fita cassete",
    "leitor de fita cassetes",
    "castelo",
    "catamarã",
    "tocador de cd",
    "violoncelo",
    "celular",
    "corrente",
    "cerca de arame",
    "corrente de malha",
    "motosserra",
    "baú",
    "chiffonier",
    "sino de vento",
    "cristaleira",
    "meia de natal",
    "igreja",
    "cinema",
    "cutelo",
    "morada do penhasco",
    "capa",
    "tamancos",
    "coqueteleira",
    "caneca de café",
    "máquina de café",
    "espiral ou bobina",
    "fechadura de combinação",
    "teclado de computador",
    "loja de doces",
    "navio porta-contentores",
    "conversível",
    "saca rolhas",
    "corneta",
    "bota cowboy",
    "chapéu de caubói",
    "berço",
    "guindaste de construção",
    "capacete de choque",
    "caixa",
    "cama infantil",
    "panela de barro",
    "bola de croquete",
    "muleta",
    "couraça",
    "barragem",
    "mesa",
    "computador de mesa",
    "telefone de discagem rotativa",
    "fralda",
    "relógio digital",
    "relogio digital",
    "mesa de jantar",
    "pano de prato",
    "lava-louças",
    "freio de disco",
    "doca",
    "trenó puxado por cães",
    "cúpula",
    "capacho",
    "equipamento de perfuração",
    "tambor",
    "baqueta",
    "haltere",
    "forno holandês",
    "ventilador elétrico",
    "guitarra elétrica",
    "locomotiva elétrica",
    "centro de entretenimento",
    "envelope",
    "máquina de espresso",
    "pó facial",
    "boá de penas",
    "armário de arquivo",
    "barco de bombeiro",
    "caminhão de bombeiros",
    "guarda-fogo",
    "mastro",
    "flauta",
    "cadeira dobrável",
    "capacete de futebol",
    "empilhadeira",
    "fonte",
    "caneta tinteiro",
    "cama com colunas",
    "carro de carga",
    "trompa francesa",
    "frigideira",
    "casaco de pele",
    "caminhão de lixo",
    "máscara de gás ou respirador",
    "bomba de gasolina",
    "cálice",
    "kart",
    "bola de golfe",
    "carrinho de golfe",
    "gôndola",
    "gongo",
    "vestido",
    "piano de cauda",
    "estufa",
    "grade do radiador",
    "bomboneria",
    "guilhotina",
    "presilha de cabelo",
    "spray cabelo",
    "meia via",
    "martelo",
    "cesto",
    "secador de cabelo",
    "computador de mão",
    "lenço",
    "drive de disco rígido",
    "harmônica",
    "harpa",
    "colheitadeira",
    "machadinha",
    "coldre",
    "home theater",
    "favo de mel",
    "gancho",
    "saia argola",
    "barra horizontal de ginástica",
    "veículo puxado por cavalos",
    "ampulheta",
    "ipod",
    "ferro de roupa",
    "abóbora esculpida",
    "jeans",
    "jipe",
    "camiseta",
    "quebra-cabeça",
    "riquixá",
    "controle de video game",
    "quimono",
    "joelheira",
    "nó",
    "bata de laboratório",
    "concha",
    "abajur",
    "notebook",
    "cortador de grama",
    "tampa da lente",
    "abridor de cartas",
    "biblioteca",
    "bote salva vidas",
    "isqueiro",
    "limusine",
    "transatlântico",
    "batom",
    "sapato de salto",
    "loção",
    "locutor de música",
    "lupa de cabeça",
    "serraria",
    "bússola magnética",
    "bolsa mensageiro",
    "caixa de correio",
    "calças",
    "maiô",
    "tampa de bueiro",
    "maracá",
    "marimbo",
    "máscara",
    "fósforo",
    "mastro",
    "labirinto",
    "copo medidor",
    "armário de remédios",
    "megalito",
    "microfone",
    "forno de micro-ondas",
    "uniforme militar",
    "lata de leite",
    "mini onibus",
    "mini-saia",
    "minivan",
    "míssil",
    "luva",
    "tigela",
    "casa móvel",
    "ford modelo t",
    "modem",
    "mosteiro",
    "monitor",
    "motocicleta",
    "almofariz e pilão",
    "chapéu de graduação",
    "mesquita",
    "mosquiteiro",
    "vespa",
    "bicicleta de montanha",
    "barraca",
    "mouse",
    "ratoeira",
    "van",
    "focinho",
    "prego de metal",
    "colar cervical",
    "colar",
    "chupeta de bebê",
    "notebook",
    "obelisco",
    "oboé",
    "ocarina",
    "odômetro",
    "filtro de óleo",
    "órgão de tubos",
    "osciloscópio",
    "overskirt",
    "carro de boi",
    "máscara de oxigênio",
    "pacote/embalagem do produto",
    "remo",
    "roda de pás",
    "cadeado",
    "pincel",
    "pijamas",
    "palácio",
    "flauta pan",
    "papel toalha",
    "paraquedas",
    "barras paralelas",
    "banco de parque",
    "parquímetro",
    "vagão de trem",
    "pátio",
    "telefone público",
    "pedestal",
    "estojo",
    "apontador de lápis",
    "perfume",
    "placa de petri",
    "fotocopiadora",
    "plectro",
    "pickelhaube",
    "cerca de piquete",
    "caminhonete",
    "cais",
    "cofrinho",
    "frasco de comprimido",
    "travesseiro",
    "bola de pingue-pongue",
    "roda",
    "navio pirata",
    "jarra de bebida",
    "plaina",
    "planetário",
    "saco de plástico",
    "prateleira",
    "arado de fazenda",
    "desentupidor",
    "câmera polaroid",
    "pólo",
    "camburão",
    "poncho",
    "mesa de bilhar",
    "garrafa de refrigerante",
    "vaso de plantas",
    "roda de oleiro",
    "furadeira elétrica",
    "tapete de oração",
    "impressora",
    "prisão",
    "míssil",
    "projetor",
    "disco de hóquei",
    "saco de pancadas",
    "bolsa",
    "quilha",
    "colcha",
    "carro de corrida",
    "raquete",
    "radiador",
    "rádio",
    "radiotelescópio",
    "barril de chuva",
    "veículo recreacional",
    "carretel de fundição de pesca",
    "câmera reflex",
    "frigorífico",
    "controle remoto",
    "restaurante",
    "revólver",
    "rifle",
    "cadeira de balanço",
    "rotisserie",
    "apagador",
    "bola de rugby",
    "vareta medidora de régua",
    "tênis",
    "seguro",
    "pino de segurança",
    "saleiro",
    "sandália",
    "sarong",
    "saxofone",
    "bainha",
    "balança",
    "ônibus escolar",
    "escuna",
    "placar",
    "monitor crt",
    "parafuso",
    "chave de fenda",
    "cinto de segurança",
    "máquina de costura",
    "escudo",
    "loja de sapatos",
    "tela shoji / divisor de quarto",
    "cesta de compras",
    "carrinho de compras",
    "pá",
    "touca de banho",
    "cortina de chuveiro",
    "esqui",
    "máscara de esqui balaclava",
    "saco de dormir",
    "regra de cálculo",
    "porta deslizante",
    "máquina caça-níqueis",
    "snorkel",
    "moto de neve",
    "limpa-neve",
    "saboneteira",
    "bola de futebol",
    "meia",
    "coletor solar térmico",
    "sombreiro",
    "tigela de sopa",
    "barra de espaço do teclado",
    "aquecedor",
    "nave espacial",
    "espátula",
    "lancha",
    "teia de aranha",
    "fuso",
    "carro esporte",
    "holofote",
    "palco",
    "locomotiva a vapor",
    "ponte em arco",
    "tambor de aço",
    "estetoscópio",
    "cachecol",
    "parede de pedra",
    "cronômetro",
    "fogão",
    "filtro",
    "bonde",
    "maca",
    "sofá",
    "estupa",
    "submarino",
    "traje",
    "relógio de sol",
    "óculos escuros",
    "óculos escuros",
    "protetor solar",
    "ponte suspensa",
    "esfregão",
    "suéter",
    "calções de banho / calções",
    "balanço",
    "interruptor elétrico",
    "seringa",
    "lâmpada de mesa",
    "tanque",
    "toca-fitas",
    "bule",
    "urso teddy",
    "televisão",
    "bola de tênis",
    "telhado de sapê",
    "cortina frontal",
    "dedal",
    "máquina de debulha",
    "trono",
    "telhado de telha",
    "torradeira",
    "tabacaria",
    "assento da privada",
    "tocha",
    "totem",
    "reboque",
    "loja de brinquedos",
    "trator",
    "caminhão semi-reboque",
    "bandeja",
    "casaco de trincheira",
    "triciclo",
    "trimarã",
    "tripé",
    "arco do triunfo",
    "trólebus",
    "trombone",
    "jacuzzi",
    "catraca",
    "teclado máquina de escrever",
    "guarda-chuva",
    "monociclo",
    "piano vertical",
    "aspirador de pó",
    "vaso",
    "teto abobadado ou arqueado",
    "tecido veludo",
    "maquina de vendas",
    "vestimenta",
    "viaduto",
    "violino",
    "vôlei",
    "máquina de waffles",
    "relógio de parede",
    "carteira",
    "guarda roupa",
    "aeronave militar",
    "pia",
    "máquina de lavar",
    "garrafa de agua",
    "jarro de água",
    "torre de água",
    "jarro de uísque",
    "assobiar",
    "peruca de cabelo",
    "tela da janela",
    "sombra de janela",
    "gravata windsor",
    "garrafa de vinho",
    "asa de avião",
    "wok",
    "colher de madeira",
    "lã",
    "cerca de trilho dividido",
    "naufrágio",
    "barco a vela",
    "yurt",
    "website",
    "história em quadrinhos",
    "palavras cruzadas",
    "sinal de trânsito ou rua",
    "semáforo",
    "capa protetora",
    "cardápio",
    "prato",
    "guacamole",
    "consomé",
    "hot pot",
    "bagatela",
    "sorvete",
    "picolé",
    "baguete",
    "bagel",
    "pretzel",
    "X-burger",
    "cachorro-quente",
    "purê de batata",
    "repolho",
    "brócolis",
    "couve-flor",
    "abobrinha",
    "espaguete de abóbora",
    "abóbora bolota",
    "abóbora",
    "pepino",
    "alcachofra",
    "pimentão",
    "cardo",
    "cogumelo",
    "maçã granny smith",
    "morango",
    "laranja",
    "limão",
    "figo",
    "abacaxi",
    "banana",
    "jaca",
    "annona cherimola",
    "romã",
    "feno",
    "carbona",
    "xarope de chocolate",
    "massa",
    "rolo de carne",
    "pizza",
    "torta de pote",
    "burrito",
    "vinho tinto",
    "expresso",
    "chávena de chá",
    "gemada alcoólica",
    "montanha",
    "bolha",
    "penhasco",
    "recife de corais",
    "géiser",
    "margem do lago",
    "promontório",
    "banco de areia",
    "praia",
    "vale",
    "vulcão",
    "jogador de baseball",
    "noivo",
    "mergulhador",
    "colza",
    "margarida",
    "cypripedium parviflorum",
    "milho",
    "bolota",
    "cinórrodo",
    "semente de castanha da Índia",
    "fungo coral",
    "agárico",
    "giromitra",
    "cogumelo chifrudo",
    "fungo estrela da terra",
    "maitake",
    "boleto",
    "espiga de milho",
    "papel higiênico"
]

# Classification

In [10]:
device = torch.device(f"cuda:{gpu}" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cuda:4


In [12]:
print(">>>>>>> Loading model")
from huggingface_hub import hf_hub_download
model_path = hf_hub_download("alefiury/CAPIVARA", "cc3m-5_captions-wyza08z5.ckpt")
if open_clip:
    if adapter is None:
        model = OpenCLIPWrapper.load_from_checkpoint(model_path, strict=False).model
    else:
        model = OpenCLIPAdapter(inference=True, devices=device)
        model.load_adapters(pretrained_adapter=adapter)
else:
    model = OpenCLIP()

model.eval()
model.to("cuda")

vision_processor = model.image_preprocessor
text_tokenizer = model.text_tokenizer

>>>>>>> Loading model


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.74 GiB total capacity; 90.13 MiB already allocated; 7.50 MiB free; 102.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model

In [48]:
images_per_class = 50 # True for ImageNet validation split
idxs = [i for i in range(1000)]
image_dataset, text_datasets, text_collate = get_data(vision_processor, text_tokenizer, imagenet_labels_pt, imagenet_prompts_pt, idxs)

image_embeddings = compute_image_embeddings(image_dataset, model)
for text_dataset in tqdm.tqdm(text_datasets, desc="Evaluating languages and prompts"):
    text_embeddings = compute_text_embeddings(text_dataset, text_collate, model)
    subset_image_embeddings_mask = [idx*images_per_class + i for idx in text_dataset.class_idxs for i in range(images_per_class)]
    acc1, acc5 = run_evaluation(
        image_embeddings[subset_image_embeddings_mask],
        text_embeddings,
        num_prompts=text_dataset.num_prompts,
        images_per_class=images_per_class,
        num_classes=len(text_dataset.class_idxs)
    )

Evaluating languages and prompts: 100%|█████████████████████████████████████████████████████████| 1/1 [00:55<00:00, 55.11s/it]


In [49]:
print(acc1, acc5)

0.4589 0.70672
